Пробуем завести Ранкер по туториалу

https://github.com/catboost/catboost/blob/master/catboost/tutorials/ranking/ranking_tutorial.ipynb

In [1]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import ndcg_score, accuracy_score

from copy import deepcopy
import numpy as np
import os
import pandas as pd

Data

In [2]:
train = pd.read_json('/kaggle/input/cl-cup-it-2023-ds/CL_Cup IT_Data_Scince___VK_/ranking_train.jsonl', lines=True)
test = pd.read_json('/kaggle/input/cl-cup-it-2023-ds/CL_Cup IT_Data_Scince___VK_/ranking_test.jsonl', lines=True)

In [3]:
def prepare_df(train, test=False):
    train = train.explode('comments')
    if not test:
        train['score'] = train['comments'].str['score']
    train['comment'] = train['comments'].str['text']
    train = train.drop(columns=['comments'])
    train['text_id'] = train.index
    return train
train = prepare_df(train)
scores = train['score'].values
test = prepare_df(test, True)
train.head(20)

,text,score,comment,text_id
0,How many summer Y Combinator fundees decided n...,0,Going back to school is not identical with giv...,0
0,How many summer Y Combinator fundees decided n...,1,There will invariably be those who don't see t...,0
0,How many summer Y Combinator fundees decided n...,2,For me school is a way to be connected to what...,0
0,How many summer Y Combinator fundees decided n...,3,I guess it really depends on how hungry you ar...,0
0,How many summer Y Combinator fundees decided n...,4,I know pollground decided to go back to school...,0
1,CBS acquires last.fm for $280m,0,It will be curious to see where this heads in ...,1
1,CBS acquires last.fm for $280m,1,Does this mean that there's now a big-name com...,1
1,CBS acquires last.fm for $280m,2,Also on BBC News: http://news.bbc.co.uk/1/low...,1
1,CBS acquires last.fm for $280m,3,I don't understand what they do that is worth ...,1
1,CBS acquires last.fm for $280m,4,sold out too cheaply. given their leadership p...,1


Embs

In [4]:
train_text_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/train_text_embs.npy' ,allow_pickle=True)
test_text_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/test_text_embs.npy' ,allow_pickle=True)
train_comment_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/train_comment_embs.npy' ,allow_pickle=True)
test_comment_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/test_comment_embs.npy' ,allow_pickle=True)

Split

In [5]:
splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 9)
split = splitter.split(train_comment_embs, groups=train['text_id'])
train_inds, test_inds = next(split)

X_train = train_comment_embs[train_inds]
X_test = train_comment_embs[test_inds]
y_train = scores[train_inds]
y_test = scores[test_inds]

Pools

In [16]:
train_pool = Pool(
    data=X_train,
    label=y_train,
    group_id=train['text_id'].values[train_inds]
)

test_pool = Pool(
    data=X_test,
    label=y_test,
    group_id=train['text_id'].values[test_inds]
)

In [41]:
default_parameters = {
    'iterations': 2000,
#     'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=10'],
    'verbose': 250,
    'random_seed': 0,
}

parameters = {}

In [18]:
def fit_model(loss_function, additional_params=None, train_pool=train_pool, test_pool=test_pool):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=True)
    
    return model

In [61]:
# model = fit_model('RMSE',{'task_type': 'GPU', 'devices': '0:1', 'custom_metric': ['PrecisionAt:top=10', 'RecallAt:top=10', 'MAP:top=10']})
# model = fit_model('RMSE', {'task_type': 'GPU', 'devices': '0:1'})
model = fit_model('YetiRankPairwise', {'task_type': 'GPU', 'devices': '0:1'}) # YetiRank YetiRankPairwise

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8232393	best: 0.8232393 (0)	total: 96.6ms	remaining: 3m 13s
250:	test: 0.8645762	best: 0.8645762 (250)	total: 18.4s	remaining: 2m 8s
500:	test: 0.8688551	best: 0.8688745 (495)	total: 36.3s	remaining: 1m 48s
750:	test: 0.8708772	best: 0.8708905 (740)	total: 54.4s	remaining: 1m 30s
1000:	test: 0.8718790	best: 0.8718978 (995)	total: 1m 12s	remaining: 1m 12s
1250:	test: 0.8725339	best: 0.8725569 (1210)	total: 1m 30s	remaining: 54.3s
1500:	test: 0.8730785	best: 0.8730785 (1500)	total: 1m 49s	remaining: 36.3s
1750:	test: 0.8736305	best: 0.8736305 (1750)	total: 2m 7s	remaining: 18.1s
1999:	test: 0.8741873	best: 0.8741873 (1999)	total: 2m 26s	remaining: 0us
bestTest = 0.8741873088
bestIteration = 1999


In [62]:
y_pred = model.predict(test_pool)
accuracy_score(y_test, np.round(y_pred))

1.1349449551696742e-05